In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
# !pip install transformers
from transformers import BertTokenizer

2023-02-06 13:34:57.880507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# None=0, Seek=1, Provide=2

df = pd.read_csv("CSN.csv")
df

,text,label
0,I don't know who left other than him.,0
1,I also was very depressed and cryed very very ...,1
2,I am a 6 yr survivor and have been out of trea...,0
3,I have enough energy for 16 hour periods.,0
4,I was diagnosed in October 2009 at age 53.,0
...,...,...
5002,"You are a creative person, and I can just imag...",0
5003,I thought they were supposed to eliminate the ...,0
5004,"I could use a good hug too, from someone who u...",1
5005,"You take 1 pill one hour prior to treatment, 1...",0


In [17]:
print(df[df['text'] == 'Add to that the fact that I just miss him, my best friend for the past 30 years, and that\'s lonely.'])
print(df[df['text'] == 'I don\'t think you are alone in your worries I too get anxious when it is getting close for check-ups.'])
print(df[df['text'] == 'At the age of 18 I was diagnosed with lung cancer'])
df.loc[df['label'] == 1, 'label'] = 0

                                                  text  label
307  Add to that the fact that I just miss him, my ...      0
                                                   text  label
1965  I don't think you are alone in your worries I ...      2
Empty DataFrame
Columns: [text, label]
Index: []


In [74]:
df

,text,label
0,I don't know who left other than him.,0
1,I also was very depressed and cryed very very ...,0
2,I am a 6 yr survivor and have been out of trea...,0
3,I have enough energy for 16 hour periods.,0
4,I was diagnosed in October 2009 at age 53.,0
...,...,...
5002,"You are a creative person, and I can just imag...",0
5003,I thought they were supposed to eliminate the ...,0
5004,"I could use a good hug too, from someone who u...",0
5005,"You take 1 pill one hour prior to treatment, 1...",0


In [84]:
df.loc[df['label'] == 2, 'label'] = 1
df.groupby('label').describe()

text                                                               
      count unique                                                top freq
label                                                                     
0      4041   4041              I don't know who left other than him.    1
1       966    966  I just wanted to let you know I was here if yo...    1

In [85]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [86]:
token = tokenizer.encode_plus(
    df['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [87]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [88]:
def generate_training_data(df_balanced, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [89]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [90]:
labels = np.zeros((len(df), 2))
labels.shape

(5007, 2)

In [91]:
labels[np.arange(len(df)), df['label'].values] = 1 # one-hot encoded target te

In [92]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [93]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [94]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [95]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [96]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [97]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [18]:
train_size

NameError: name 'train_size' is not defined

In [98]:
train_dataset

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 2), dtype=tf.float64, name=None))>

In [99]:
from transformers import TFBertModel

In [100]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [101]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [102]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [103]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [104]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2


2023-01-21 20:53:36.578018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - ETA: 0s - loss: 0.2727 - accuracy: 0.8830

2023-01-21 20:58:51.602657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


249/249 [==============================] - 343s 1s/step - loss: 0.2727 - accuracy: 0.8830 - val_loss: 0.1930 - val_accuracy: 0.9266
Epoch 2/2
249/249 [==============================] - 326s 1s/step - loss: 0.1757 - accuracy: 0.9305 - val_loss: 0.1062 - val_accuracy: 0.9663


In [105]:
sentiment_model.save('sentiment_model')

INFO:tensorflow:Assets written to: sentiment_model/assets


INFO:tensorflow:Assets written to: sentiment_model/assets


In [106]:
sentiment_model = tf.keras.models.load_model('sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Non empathetic', 'Empathetic']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]
#     return probs

In [107]:
input_text = input('Enter sentence here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter sentence here: ki haal chaal kudiye


2023-01-21 21:11:04.660144: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step
Predicted Sentiment: Non empathetic


In [108]:
df_set = pd.read_csv("test.csv")

In [109]:
seek = []
seek_label = []
for i in df_set["response_post"]:
    processed_data = prepare_data(i, tokenizer)
    result = make_prediction(sentiment_model, processed_data=processed_data)
    seek.append(i)
    seek_label.append(result)

1/1 [==============================] - 0s 107ms/step


1/1 [==============================] - 0s 105ms/step


1/1 [==============================] - 0s 110ms/step


1/1 [==============================] - 0s 106ms/step


1/1 [==============================] - 0s 105ms/step


...

1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 113ms/step


In [110]:
final_seek = pd.DataFrame()
final_seek["respense_post"] = seek
final_seek["empathetic?"] = seek_label

In [111]:
final_seek

,respense_post,empathetic?
0,"That's pretty vague, do you not know what you'...",Non empathetic
1,idk what a Red pill means exactly but my advic...,Empathetic
2,"I think it's social anxiety , that creates par...",Non empathetic
3,By any chance do you think you're in a loop. J...,Non empathetic
4,depression. not sadness which is caused by som...,Non empathetic
...,...,...
9247,"All day, every day. It's definitely not just y...",Empathetic
9248,like being stuck in a black hole. At times you...,Non empathetic
9249,I probably would have considered bringing harm...,Non empathetic
9250,People barely notice me too,Non empathetic


In [112]:
final_seek.to_csv('final_response_mhrt.csv', encoding='utf-8')

In [114]:
final_seek.groupby('empathetic?').describe()

respense_post         \
                       count unique   
empathetic?                           
Empathetic              3508   1340   
Non empathetic          5744   2124   

                                                                        
                                                              top freq  
empathetic?                                                             
Empathetic      but how can you play sims when you're dead? jo...    9  
Non empathetic  I almost broke down crying just then for liter...    6